# Gather Insights from Industrial Data Stores with Amazon Bedrock

## Introduction

In this notebook, you will learn how to build a natural language interface for manufacturing data using Amazon Bedrock's Converse API. This powerful approach allows you to query complex industrial databases without writing SQL or understanding database schemas.

**Key Learning Objectives:**
1. How to connect foundation models to industrial data sources
2. How to use Amazon Bedrock's Converse API with tools
3. How to build a system that translates natural language to SQL
4. How to create a complete workflow from user question to informative answer
5. How to easily switch between different AI models without changing your application code

### Context

Large language models (LLMs) excel at reasoning and understanding natural language, but they need a way to access your enterprise data to be truly helpful. Rather than trying to directly feed database information to an LLM (which can lead to hallucinations), we can leverage their tool-calling capabilities to:

1. **Understand** a user's question
2. **Plan** what data is needed to answer it
3. **Generate** appropriate database queries
4. **Interpret** the results to provide a helpful response

This approach makes industrial data accessible to everyone in your organization through natural language, eliminating the need for:
- SQL expertise
- Knowledge of database schemas
- Specialized database tools
- Training on specific systems

### Manufacturing Execution System (MES) Overview

In this workshop, you'll work with a simulated Manufacturing Execution System (MES) that tracks the production of electric bicycles. Our MES database includes:

- **Products**: E-bikes and their components
- **Work Orders**: Production jobs with schedules and status
- **Inventory**: Raw materials and components tracking  
- **Machines**: Equipment specifications and maintenance schedules
- **Quality Control**: Inspection results with defect details
- **Employees**: Personnel records with roles and skills
- **And more**: Work centers, downtime events, OEE metrics

The patterns you learn can be applied to any industrial data source, whether it's a different MES system, a data historian, an ERP system, or another industrial database.

## Prerequisites

Before getting started, make sure you have the following:

- Python 3.x installed
- Access to an AWS account with permissions to use Amazon Bedrock
- Model access enabled for at least one of the following:
  - Anthropic Claude models (Claude 3 Haiku, Claude 3 Sonnet)
  - Amazon Nova models 
- The SQLite database created in the previous step of the workshop

If running on Amazon SageMaker AI Studio, a `ml.t3.medium` instance is sufficient for this notebook.

## Environment Setup

First, let's install the required packages:

In [22]:
# Install requirements (only needs to be run once)
%pip install -q -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Load environment variables if needed (this is optional and not required in SageMaker Studio):

In [1]:
# Environment setup - only needed if not running in SageMaker Studio
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

## Setting up the Amazon Bedrock Client

Amazon Bedrock provides a unified API for accessing foundation models from various providers. We'll use the boto3 client for bedrock-runtime to access these models:

In [2]:
import boto3
import json
import time
from datetime import datetime

def get_bedrock_client():
    """
    Create a bedrock-runtime client for model inference
    
    Returns:
        boto3.client: A configured Bedrock runtime client
    """
    return boto3.client(
        service_name='bedrock-runtime',
        region_name=os.getenv("AWS_REGION", "us-east-1"),
        endpoint_url=f'https://bedrock-runtime.{os.getenv("AWS_REGION", "us-east-1")}.amazonaws.com',
    )

# Create the client
bedrock_client = get_bedrock_client()

Let's test our Bedrock setup with a simple prompt. This will confirm that:
1. Our AWS credentials are valid
2. We have access to the specified model
3. The client is configured correctly

In [3]:
def test_bedrock_model(prompt, model_id="anthropic.claude-3-haiku-20240307-v1:0"):
    """
    Test the Bedrock model with a simple prompt
    
    Args:
        prompt (str): The text prompt to send to the model
        model_id (str): The Bedrock model identifier
        
    Returns:
        str: The model's response text
    """
    # Different model families need different request formats
    if "anthropic" in model_id:
        body = json.dumps({
            "max_tokens": 1024,
            "temperature": 0.5,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "anthropic_version": "bedrock-2023-05-31"
        })
    elif "nova" in model_id:
        body = json.dumps({
            "schemaVersion": "messages-v1",
            "messages": [
                {"role": "user", "content": [{"text": prompt}]}
            ],
            "inferenceConfig": {
                "maxTokens": 1024,
                "temperature": 0.5,
                "topP": 0.9
            }
        })
    elif "amazon" in model_id:
        body = json.dumps({
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 1024,
                "temperature": 0.5,
                "topP": 0.9
            }
        })
    
    # Call the model
    response = bedrock_client.invoke_model(
        body=body, 
        modelId=model_id
    )
    
    # Parse the response based on the model family
    response_body = json.loads(response.get("body").read())
    
    if "anthropic" in model_id:
        return response_body['content'][0]['text']
    elif "nova" in model_id:
        # Nova response format is different
        if 'contentBlocks' in response_body and len(response_body['contentBlocks']) > 0:
            # Extract text from content blocks
            content_block = response_body['contentBlocks'][0]
            if 'text' in content_block:
                return content_block['text']
            elif 'content' in content_block and len(content_block['content']) > 0:
                return content_block['content'][0].get('text', '')
        return str(response_body)
    elif "amazon" in model_id:
        return response_body.get('results')[0].get('outputText')



In [4]:
# Test with a simple prompt
prompt = "Hello, can you tell me about manufacturing execution systems (MES) in 2-3 sentences?"
response = test_bedrock_model(prompt)
print("Claude 3 Haiku Response:")
print(response)

Claude 3 Haiku Response:
Certainly! Here's a brief overview of manufacturing execution systems (MES) in 2-3 sentences:

Manufacturing execution systems (MES) are software solutions that help manufacturers optimize their production processes by providing real-time visibility, control, and coordination of operations on the factory floor. MES systems integrate with other enterprise systems, such as ERP (enterprise resource planning) and SCADA (supervisory control and data acquisition), to streamline production, improve quality, and enhance overall operational efficiency. They are widely used in various industries, including automotive, aerospace, electronics, and pharmaceuticals, to enhance productivity, reduce costs, and ensure compliance with regulatory requirements.


## Comparing Different Models

One of the challenges in working with foundation models is that different models have their own response formats, capabilities, and performance characteristics. Let's compare several models on the same question to see these differences:

In [5]:
# Compare responses from different models
import time

# Define models to test
models = [
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "us.amazon.nova-micro-v1:0"
]

prompt = "Tell me about the top 3 trends in Industrial Manufacturing in 3 bullet points"

# Test each model
for model_id in models:
    print(f"\nTesting {model_id}...")
    start_time = time.time()
    
    try:
        response = test_bedrock_model(prompt, model_id)
        end_time = time.time()
        execution_time = end_time - start_time
        
        print(f"Response:\n{response}")
        print(f"\nExecution time: {execution_time:.2f} seconds")
    except Exception as e:
        print(f"Error with model {model_id}: {e}")
        print("Please make sure you have enabled access to this model in the AWS Bedrock console.")


Testing anthropic.claude-3-haiku-20240307-v1:0...
Response:
The top 3 trends in Industrial Manufacturing are:

- Automation and Robotics: The increased adoption of advanced automation technologies, such as industrial robots, cobots (collaborative robots), and AI-powered systems, to enhance productivity, efficiency, and precision in manufacturing processes.

- Digitalization and Industry 4.0: The integration of digital technologies, including the Internet of Things (IoT), cloud computing, data analytics, and simulation, to create smart, connected, and data-driven manufacturing environments, enabling real-time decision-making and optimization.

- Sustainability and Circular Economy: The focus on sustainable manufacturing practices, such as the use of renewable energy, recycling, and the adoption of circular economy principles, to reduce environmental impact, minimize waste, and promote the efficient use of resources.

Execution time: 2.19 seconds

Testing anthropic.claude-3-sonnet-20240

**Key Observations:**

1. **Response Quality**: Different models produce different responses, even to the same question
2. **Response Time**: Larger models (Sonnet) typically take longer than smaller models (Haiku)
3. **Response Format**: Each model family returns a different JSON structure, which we had to handle separately

This highlights a challenge in production applications: switching between models can require significant code changes to handle different request and response formats. Later, we'll see how the Bedrock Converse API solves this problem by providing a consistent interface.

## Database Exploration

Before building our natural language interface, let's explore the MES database to understand its structure. This exploration helps us understand what data we have access to and how it's organized.

In [6]:
# First, let's create a helper function to execute SQL queries against our SQLite database
import sqlite3
import pandas as pd

def query_sqlite(query, db_path="mes.db"):
    """
    Executes a SQL query against a SQLite database and returns the results as a pandas DataFrame
    
    Parameters
    ----------
    query : str
        An SQL query to execute
    db_path : str
        Path to the SQLite database file
        
    Returns
    ----------
    pandas.DataFrame
        The results of the SQL query
    """
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except Exception as e:
        return f"Error executing query: {str(e)}"

First, let's check if the database exists and verify what tables it contains:

In [7]:
# Check if the database file exists
import os
if not os.path.exists('mes.db'):
    print("MES database not found. We need to create it first.")
    print("Running the synthetic data generator...")
    
    # Run the generator script to create the database
    !python MES-synthetic-data/sqlite-synthetic-mes-data.py

# Show tables in the database
print("Tables in the database:")
tables = query_sqlite("SELECT name FROM sqlite_master WHERE type='table';")
print(tables)

Tables in the database:
                   name
0              Products
1             Suppliers
2           WorkCenters
3                Shifts
4             Inventory
5              Machines
6             Employees
7       BillOfMaterials
8            WorkOrders
9            OEEMetrics
10            Downtimes
11       QualityControl
12  MaterialConsumption
13              Defects


Now that we've confirmed the database exists, let's examine each table's structure and sample data:

In [8]:
# Function to explore table structure and sample data
def explore_table(table_name, limit=5):
    """
    Explores a table's structure and sample data
    
    Parameters
    ----------
    table_name : str
        The name of the table to explore
    limit : int
        Number of sample rows to display
        
    Returns
    ----------
    None
        Prints table info to the console
    """
    # Get table schema
    schema = query_sqlite(f"PRAGMA table_info({table_name});")
    
    # Get sample data
    sample_data = query_sqlite(f"SELECT * FROM {table_name} LIMIT {limit};")
    
    print(f"\n--- {table_name} Table ---")
    print("\nSchema:")
    print(schema[['name', 'type']])
    
    print("\nSample Data:")
    print(sample_data)
    
    # Get row count
    count = query_sqlite(f"SELECT COUNT(*) as count FROM {table_name};")
    print(f"\nTotal Rows: {count['count'].values[0]}")
    
    print("-" * 50)

In [9]:
# Explore each table in the database
# Note: You may want to run only a few tables at a time to avoid long output
# For this demo, we'll limit to just a few important tables
key_tables = ['Products', 'WorkOrders', 'Machines', 'Inventory', 'QualityControl']

for table_name in tables['name']:
    if table_name in key_tables:
        explore_table(table_name)


--- Products Table ---

Schema:
                  name     type
0            ProductID  INTEGER
1                 Name  VARCHAR
2          Description  VARCHAR
3             Category  VARCHAR
4                 Cost    FLOAT
5  StandardProcessTime    FLOAT
6             IsActive  BOOLEAN

Sample Data:
   ProductID        Name                                        Description  \
0          1  eBike T101  eBike T101: High-performance electric bicycle ...   
1          2  eBike T200  eBike T200: Premium mountain e-bike with rugge...   
2          3  eBike C150  eBike C150: Comfortable city commuter e-bike w...   
3          4  eBike M300  eBike M300: Mid-drive electric mountain bike w...   
4          5       Forks  Forks: Durable suspension forks for smooth rid...   

         Category    Cost  StandardProcessTime  IsActive  
0  Electric Bikes  739.47                 4.55         1  
1  Electric Bikes  452.90                 6.07         1  
2  Electric Bikes   66.38                 4.9

This database exploration gives us valuable insights into what data is available. In a real application, we would:

1. Understand table relationships (e.g., WorkOrders reference Products via ProductID)
2. Identify key metrics and KPIs (e.g., defect rates, machine utilization)
3. Understand data formats and constraints

All of this knowledge would typically need to be provided to users through documentation or training. However, with our natural language interface, we'll instead provide this information to the AI model, which will use it to answer user questions.

## Understanding the Bedrock Converse API with Tools

The **Bedrock Converse API** is a powerful feature that allows foundation models to interact with external tools and data sources. This makes it ideal for our use case of querying a database.

Here's how tool use works with the Converse API:

1. A user asks a question
2. We pass the question to the model through the Converse API
3. The model decides if it needs to use a tool to answer
4. If a tool is needed, the model requests to call the tool with specific parameters
5. We execute the tool and return the results to the model
6. The model incorporates the tool results to generate a final answer

This process can involve multiple tool calls within a single conversation turn, allowing the model to gather all the information needed to answer the question.

Let's define our tool configurations for interacting with the MES database:

In [10]:
# Define tool configurations for interacting with the MES database
def get_tool_config():
    """
    Get the tool configuration for the Bedrock Converse API
    
    Returns
    -------
    dict
        Tool configuration for the Converse API
    """
    return {
        "tools": [
            {
                "toolSpec": {
                    "name": "get_schema",
                    "description": "ALWAYS use this tool FIRST to get the schema of the MES database before attempting any SQL queries. This provides details about all tables, columns, relationships, and sample data.",
                    "inputSchema": {
                        "json": {
                            "type": "object",
                            "properties": {}
                        }
                    }
                }
            },
            {
                "toolSpec": {
                    "name": "execute_sql",
                    "description": "Execute SQL queries against the MES database ONLY after you have retrieved and examined the schema. Write efficient SQL that joins relevant tables and focuses on answering the user's specific question.",
                    "inputSchema": {
                        "json": {
                            "type": "object",
                            "properties": {
                                "sql_query": {
                                    "type": "string",
                                    "description": "The SQL query to execute against the MES database. Write clean, efficient SQL that joins necessary tables to answer the user's question in one query when possible."
                                }
                            },
                            "required": [
                                "sql_query"
                            ]
                        }
                    }
                }
            }
        ]
    }

Our tool configuration defines two tools:

1. **get_schema**: This tool retrieves the database schema, including all tables, columns, relationships, and sample data. This helps the model understand what data is available and how to query it.

2. **execute_sql**: This tool executes SQL queries against the MES database. The model will use this tool to retrieve the specific data needed to answer the user's question.

Now, let's implement the functionality for these tools:

In [11]:
# DatabaseQueryTool class to handle SQL queries and schema retrieval
class DatabaseQueryTool:
    """A tool for executing SQL queries against the MES database"""
    
    def __init__(self, db_path="mes.db"):
        """Initialize with the database path"""
        self.db_path = db_path
        self._schema_cache = None
        self._schema_cache_time = None
        self._cache_expiry = 60 * 5  # Cache expires after 5 minutes
    
    def execute_query(self, sql_query):
        """
        Execute a SQL query and return the results
        
        Args:
            sql_query (str): The SQL query to execute
            
        Returns:
            dict: Query results or error information
        """
        print(f"Executing SQL query: {sql_query}")
        start_time = time.time()
        
        try:
            # Connect to the database
            conn = sqlite3.connect(self.db_path)
            
            # Execute the query
            df = pd.read_sql_query(sql_query, conn)
            conn.close()
            
            # Process datetime columns for better display
            for col in df.columns:
                if df[col].dtype == 'object':
                    # Try to convert string columns that might be dates
                    try:
                        if df[col].str.contains('-').any() and df[col].str.contains(':').any():
                            df[col] = pd.to_datetime(df[col])
                            # Format datetime for display
                            df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M')
                    except:
                        pass
            
            # Round float columns to 2 decimal places for display
            for col in df.select_dtypes(include=['float']).columns:
                df[col] = df[col].round(2)
            
            # Convert to JSON-serializable format
            result = {
                "success": True,
                "rows": df.to_dict(orient="records"),
                "column_names": df.columns.tolist(),
                "row_count": len(df),
                "execution_time_ms": round((time.time() - start_time) * 1000, 2)
            }
            
            print(f"Query executed successfully: {len(df)} rows returned in {result['execution_time_ms']}ms")
            return result
            
        except Exception as e:
            error_msg = str(e)
            print(f"Error executing SQL query: {error_msg}")
            
            # Provide more helpful error messages for common issues
            if "no such table" in error_msg.lower():
                table_name = error_msg.split("no such table:", 1)[1].strip() if "no such table:" in error_msg else "unknown"
                error_msg = f"Table '{table_name}' doesn't exist. Please check the schema and table names."
            elif "no such column" in error_msg.lower():
                col_name = error_msg.split("no such column:", 1)[1].strip() if "no such column:" in error_msg else "unknown"
                error_msg = f"Column '{col_name}' doesn't exist. Please check the schema and column names."
            elif "syntax error" in error_msg.lower():
                error_msg = f"SQL syntax error: {error_msg}. Please check your query syntax."
            
            return {
                "success": False,
                "error": error_msg,
                "execution_time_ms": round((time.time() - start_time) * 1000, 2)
            }
    
    def get_schema(self):
        """
        Get the database schema with caching for performance
        
        Returns:
            dict: Comprehensive schema information
        """
        current_time = time.time()
        
        # Return cached schema if available and fresh
        if (self._schema_cache is not None and 
            self._schema_cache_time is not None and 
            current_time - self._schema_cache_time < self._cache_expiry):
            print("Returning cached schema")
            return self._schema_cache
        
        print("Retrieving fresh database schema")
        start_time = time.time()
        
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Get all tables
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()
            
            schema = {}
            for table in tables:
                table_name = table[0]
                
                # Get column information
                cursor.execute(f"PRAGMA table_info({table_name});")
                columns = cursor.fetchall()
                
                # Format column information
                column_info = []
                for col in columns:
                    column_info.append({
                        "name": col[1],
                        "type": col[2],
                        "notnull": bool(col[3]),
                        "pk": bool(col[5])
                    })
                
                # Get foreign key relationships
                cursor.execute(f"PRAGMA foreign_key_list({table_name});")
                foreign_keys = cursor.fetchall()
                
                fk_info = []
                for fk in foreign_keys:
                    fk_info.append({
                        "id": fk[0],
                        "seq": fk[1],
                        "table": fk[2],
                        "from": fk[3],
                        "to": fk[4]
                    })
                
                # Get table row count
                cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
                row_count = cursor.fetchone()[0]
                
                # Get sample data (limited to 3 rows for performance)
                cursor.execute(f"SELECT * FROM {table_name} LIMIT 3;")
                sample_data = cursor.fetchall()
                
                # Get column names for the sample data
                column_names = [col[1] for col in columns]
                
                # Format sample data as records
                sample_data_records = []
                for row in sample_data:
                    record = {}
                    for i, value in enumerate(row):
                        record[column_names[i]] = value
                    sample_data_records.append(record)
                
                # Add table information to schema
                schema[table_name] = {
                    "columns": column_info,
                    "foreign_keys": fk_info,
                    "row_count": row_count,
                    "sample_data": sample_data_records
                }
            
            # Add schema metadata
            schema["__metadata__"] = {
                "database_name": self.db_path.split("/")[-1],
                "total_tables": len(tables),
                "generated_at": datetime.now().isoformat(),
                "schema_version": "1.1"
            }
            
            conn.close()
            
            # Update cache
            self._schema_cache = schema
            self._schema_cache_time = current_time
            
            print(f"Schema retrieved in {round((time.time() - start_time) * 1000, 2)}ms")
            return schema
            
        except Exception as e:
            print(f"Error retrieving schema: {e}")
            return {
                "error": f"Failed to retrieve schema: {str(e)}",
                "timestamp": datetime.now().isoformat()
            }

# Initialize the database tool
db_tool = DatabaseQueryTool()

Now that we have our tool implementation, we need a function to handle tool requests from the model. This function will:

1. Receive tool requests from the model
2. Execute the appropriate tool function
3. Return the results to the model
4. Continue the conversation

In [12]:
def handle_tool_request(tool, model_id, conversation, query_timeout=60):
    """
    Handle tool requests from the model
    
    Parameters
    ----------
    tool : dict
        The tool request from the model
    model_id : str
        The model ID to use for the conversation
    conversation : list
        The conversation history
    query_timeout : int
        Timeout for SQL queries in seconds
        
    Returns
    -------
    tuple
        (response, conversation, tool_response)
    """
    tool_use = tool["toolUse"]
    tool_use_id = tool_use["toolUseId"]
    tool_name = tool_use["name"]
    
    print(f"Tool request received: {tool_name}, ID: {tool_use_id}")
    
    # Store tool responses for later display in the assistant message
    tool_response = {
        "type": tool_name,
        "data": None
    }
    
    # Execute the appropriate tool
    if tool_name == "execute_sql":
        sql_query = tool_use["input"]["sql_query"]
        
        # Save SQL for display
        tool_response["sql_query"] = sql_query
        
        # Execute the SQL query
        start_time = time.time()
        result = db_tool.execute_query(sql_query)
        elapsed_time = time.time() - start_time
        
        if result["success"]:
            tool_response["success"] = True
            tool_response["execution_time"] = elapsed_time
            tool_response["row_count"] = result["row_count"]
            
            # Convert to dataframe for display
            if result["row_count"] > 0:
                df = pd.DataFrame(result["rows"])
                tool_response["dataframe"] = df
                
                # Display the query results
                print(f"\nQuery results ({result['row_count']} rows):")
                print(df.head().to_string())
                if result["row_count"] > 5:
                    print(f"...and {result['row_count'] - 5} more rows")
            
            # Prepare the tool result response
            tool_result = {
                "toolUseId": tool_use_id,
                "content": [{"json": result}]
            }
        else:
            tool_response["success"] = False
            tool_response["error"] = result["error"]
            tool_response["execution_time"] = elapsed_time
            
            # Prepare the error response
            tool_result = {
                "toolUseId": tool_use_id,
                "content": [{"text": f"Error executing SQL: {result['error']}"}],
                "status": "error"
            }
    
    elif tool_name == "get_schema":
        # Get the database schema
        schema = db_tool.get_schema()
        
        # Save schema info for display
        # Filter out metadata entry when counting columns
        total_tables = len([k for k in schema.keys() if k != "__metadata__"])
        total_columns = sum(len(table_info.get("columns", [])) 
                          for table_name, table_info in schema.items() 
                          if table_name != "__metadata__")
        
        tool_response["data"] = {
            "total_tables": total_tables,
            "total_columns": total_columns,
            "schema": schema
        }
        
        print(f"\nSchema retrieved: {total_tables} tables, {total_columns} columns")
        
        # Prepare the tool result response
        tool_result = {
            "toolUseId": tool_use_id,
            "content": [{"json": schema}]
        }
    
    else:
        # Unknown tool
        print(f"Unknown tool requested: {tool_name}")
        
        tool_response["success"] = False
        tool_response["error"] = f"Unknown tool: {tool_name}"
        
        tool_result = {
            "toolUseId": tool_use_id,
            "content": [{"text": f"Unknown tool: {tool_name}"}],
            "status": "error"
        }
    
    # Add the tool result to the conversation
    tool_result_message = {
        "role": "user",
        "content": [
            {
                "toolResult": tool_result
            }
        ]
    }
    conversation.append(tool_result_message)
    
    # Send the tool result to the model
    response = bedrock_client.converse(
        modelId=model_id,
        messages=conversation,
        toolConfig=get_tool_config(),
        inferenceConfig={
            "maxTokens": 4096,
            "temperature": 0.1
        }
    )
    
    return response, conversation, tool_response

## Building the MES Question Answering System

Now we're ready to put all the pieces together to build our complete natural language interface for the MES database. The main function will:

1. Take a user's question about the MES data
2. Send it to the model with appropriate system instructions
3. Handle any tool requests (getting schema, executing SQL)
4. Return the final answer

The function follows this process:
1. Receives a user question and initializes the conversation
2. Sends the initial request to the model with system instructions
3. Checks if the model wants to use a tool (like getting schema or running a query)
4. If yes, handles the tool request and sends the results back to the model
5. Repeats steps 3-4 until the model provides a final answer
6. Returns the final answer along with all intermediate steps

Let's implement this function:

In [13]:
def ask_mes_question(question, model_id="anthropic.claude-3-haiku-20240307-v1:0", temperature=0.1):
    """
    Ask a question about the MES using the Bedrock Converse API with tools
    
    Parameters
    ----------
    question : str
        The question to ask about the MES
    model_id : str
        The model ID to use for the conversation
    temperature : float
        The temperature to use for the model
        
    Returns
    -------
    dict
        A dictionary containing the question, response, and any tool outputs
    """
    print(f"Question: {question}")
    print("-" * 50)
    
    # Create system prompt
    system_prompt = """You are an expert manufacturing analyst for a Manufacturing Execution System (MES) for an e-bike manufacturing facility.

Your role is to help users extract insights by querying the MES database that tracks:
- Products (e-bikes, components, and parts)
- Work Orders (production jobs with schedules and status)
- Inventory (raw materials, components, and stock levels)
- Work Centers (manufacturing areas like Frame Fabrication, Wheel Production)
- Machines (equipment with efficiency metrics and maintenance records)
- Quality Control (inspection results, defects, and yield rates)
- Material Consumption (component usage tracking)
- Downtime Events (machine issues and reasons)
- OEE Metrics (Overall Equipment Effectiveness measurements)
- Employees (operators, technicians, and managers)

IMPORTANT GUIDELINES:
1. ALWAYS use the get_schema tool FIRST to understand the database structure.
2. Write efficient SQLite queries - prefer JOINs to retrieve related data in a single query. When manipulating dateime, you should use something like date('now', '-1 month').
3. For questions about trends or patterns, include visualizable metrics.
4. For inventory questions, consider reorder levels and stock status.
5. For quality questions, look at defect types and rates.
6. For machine questions, consider OEE metrics and maintenance schedules.
7. For production questions, consider work order status and schedule adherence.

FORMAT YOUR RESPONSES:
1. First, briefly restate what you understood from the question
2. Present a concise summary of the key findings
3. Add relevant details or observations beneath your summary
4. If applicable, suggest follow-up questions the user might want to ask

Keep your explanations clear and relevant to manufacturing operations. Avoid excessive technical jargon when explaining results.
"""
    
    # Start timer
    start_time = time.time()
    
    # Create user message
    user_message = {
        "role": "user",
        "content": [{"text": question}]
    }
    
    # Initialize conversation
    conversation = [user_message]
    
    # First model call
    response = bedrock_client.converse(
        modelId=model_id,
        messages=conversation,
        system=[{"text": system_prompt}],
        toolConfig=get_tool_config(),
        inferenceConfig={
            "maxTokens": 4096,
            "temperature": temperature
        }
    )
    
    # Store tool responses
    tool_responses = []
    
    # Handle tool use requests as needed
    stop_reason = response["stopReason"]
    
    while stop_reason == "tool_use":
        # Get the tool request
        tool_requests = response["output"]["message"]["content"]
        
        # Add the assistant message to the conversation
        conversation.append(response["output"]["message"])
        
        # Process each tool request
        for tool_request in tool_requests:
            if "toolUse" in tool_request:
                # Handle the tool request
                response, conversation, tool_response = handle_tool_request(
                    tool_request, model_id, conversation
                )
                
                # Store the tool response
                tool_responses.append(tool_response)
                
                # Check if we need to process another tool request
                stop_reason = response["stopReason"]
    
    # Extract the final text response
    final_message = response["output"]["message"]
    conversation.append(final_message)
    
    # Display the final text response
    final_text = ""
    for content_block in final_message["content"]:
        if "text" in content_block:
            final_text += content_block["text"]
    
    # Add elapsed time
    elapsed_time = round(time.time() - start_time, 2)
    
    print("-" * 50)
    print(f"Response (completed in {elapsed_time}s):")
    print(final_text)
    
    # Return everything for further analysis if needed
    return {
        "question": question,
        "response": final_text,
        "tool_responses": tool_responses,
        "conversation": conversation,
        "elapsed_time": elapsed_time
    }

Now let's test our MES question answering system by asking some real questions about our manufacturing data:

In [14]:
# Let's test our system with a simple question about machines
result = ask_mes_question("What is the status of each machine in the facility?")

Question: What is the status of each machine in the facility?
--------------------------------------------------
Tool request received: get_schema, ID: tooluse_WtOoPxBcRMOBEpj9wGhK-w
Retrieving fresh database schema
Schema retrieved in 34.66ms

Schema retrieved: 14 tables, 127 columns
Tool request received: execute_sql, ID: tooluse_FfGDhnsmTVyTc8VakSYkrA
Executing SQL query: SELECT 
  m.Name AS Machine,
  m.Status,
  m.NextMaintenanceDate,
  m.LastMaintenanceDate,
  m.NominalCapacity,
  m.CapacityUOM
FROM Machines m
Query executed successfully: 14 rows returned in 17.08ms

Query results (14 rows):
          Machine       Status NextMaintenanceDate LastMaintenanceDate  NominalCapacity     CapacityUOM
0  Machine Fra-10  maintenance    2025-03-07 20:38    2025-02-28 10:38             7.40     frames/hour
1  Machine Fra-11      running    2025-03-10 14:38    2025-03-02 10:38             9.31     frames/hour
2  Machine Whe-20      running    2025-01-21 08:38    2025-01-13 10:38            2

As you can see, the model:
1. First used the `get_schema` tool to understand the database structure
2. Then used the `execute_sql` tool to query the Machines table
3. Finally, it provided a clear, human-readable answer to the question

This demonstrates the power of the Converse API with tools - it allows the model to access external data sources when needed, making its responses accurate and up-to-date.

Let's try a more complex question that might require joining multiple tables:

In [15]:
# Question about work orders and products
result = ask_mes_question("Show me all completed work orders for eBike products")

Question: Show me all completed work orders for eBike products
--------------------------------------------------
Tool request received: get_schema, ID: tooluse_Bo8jy0kGRw-K0xCijoQKcQ
Returning cached schema

Schema retrieved: 14 tables, 127 columns
Tool request received: execute_sql, ID: tooluse_Bf1L4DKIT8CVuzkGNBUZmw
Executing SQL query: SELECT 
  wo.OrderID,
  p.Name AS ProductName,
  wo.Quantity,
  wo.ActualProduction,
  wo.ActualStartTime,
  wo.ActualEndTime
FROM WorkOrders wo
JOIN Products p ON wo.ProductID = p.ProductID
WHERE wo.Status = 'completed' AND p.Category = 'Electric Bikes'
Query executed successfully: 20 rows returned in 7.82ms

Query results (20 rows):
   OrderID ProductName  Quantity  ActualProduction   ActualStartTime     ActualEndTime
0        3  eBike M300        32                32  2025-02-04 18:07  2025-02-04 19:44
1       10  eBike M300        23                23  2025-03-01 04:23  2025-03-01 05:15
2       12  eBike T101        25                25  2024-12-

Notice how the model was able to:
1. Identify that it needed to join the WorkOrders and Products tables
2. Understand that products with names like "eBike" are what we're looking for
3. Filter for completed work orders
4. Present a clear summary of the results

Now, let's try a question that requires understanding time-based filters:

In [16]:
# Question with time component
result = ask_mes_question("What work orders were completed in the past month?")

Question: What work orders were completed in the past month?
--------------------------------------------------
Tool request received: execute_sql, ID: tooluse_Ds1sNtx0SSCBaY_MDFuTnA
Executing SQL query: SELECT 
  wo.work_order_id,
  wo.product_id,
  wo.start_date,
  wo.end_date
FROM work_orders wo
WHERE wo.end_date >= date('now', '-1 month')
  AND wo.end_date <= date('now')
ORDER BY wo.end_date DESC;
Error executing SQL query: Execution failed on sql 'SELECT 
  wo.work_order_id,
  wo.product_id,
  wo.start_date,
  wo.end_date
FROM work_orders wo
WHERE wo.end_date >= date('now', '-1 month')
  AND wo.end_date <= date('now')
ORDER BY wo.end_date DESC;': no such table: work_orders
Tool request received: get_schema, ID: tooluse_z-zqZEhqTwGQktWyJer64A
Returning cached schema

Schema retrieved: 14 tables, 127 columns
Tool request received: execute_sql, ID: tooluse_Mr6GMDjCTsuIj5Y91hYszg
Executing SQL query: SELECT 
  wo.OrderID,
  wo.ProductID,
  wo.ActualStartTime,
  wo.ActualEndTime
FROM W

The model understands concepts like "the past month" and translates them into appropriate SQL date filters. It also knows to join the relevant tables to provide context about the products being manufactured.

## Model Comparison with Converse

One of the key advantages of using Amazon Bedrock is the ability to easily switch between different foundation models. Let's create a function to compare how different models handle the same question:

In [17]:
def compare_models(question, models=None):
    """
    Compare different models on the same question
    
    Parameters
    ----------
    question : str
        The question to ask
    models : list
        List of model IDs to compare
        
    Returns
    -------
    dict
        Dictionary of results by model
    """
    if models is None:
        models = [
            "anthropic.claude-3-haiku-20240307-v1:0",
            "anthropic.claude-3-sonnet-20240229-v1:0",
            "us.amazon.nova-lite-v1:0"
        ]
    
    results = {}
    
    for model_id in models:
        print(f"\n\n{'='*80}")
        print(f"Testing model: {model_id}")
        print(f"{'='*80}\n")
        
        try:
            result = ask_mes_question(question, model_id=model_id)
            results[model_id] = result
        except Exception as e:
            print(f"Error with model {model_id}: {e}")
            print("Make sure you have enabled access to this model in the AWS Bedrock console.")
            results[model_id] = {"error": str(e)}
    
    return results

Let's compare how different models handle a question about inventory management:

In [18]:
# Let's compare models on an inventory-related question
comparison = compare_models("Which inventory items are below their reorder level?")



Testing model: anthropic.claude-3-haiku-20240307-v1:0

Question: Which inventory items are below their reorder level?
--------------------------------------------------
Tool request received: get_schema, ID: tooluse_KLsK7ndBRlKk-mcozulXZw
Returning cached schema

Schema retrieved: 14 tables, 127 columns
Tool request received: execute_sql, ID: tooluse_zsxLos4IR5OoHll2jnEbew
Executing SQL query: SELECT i.Name, i.Quantity, i.ReorderLevel
FROM Inventory i
WHERE i.Quantity < i.ReorderLevel
Query executed successfully: 1 rows returned in 3.71ms

Query results (1 rows):
            Name  Quantity  ReorderLevel
0  Ball Bearings        69            89
--------------------------------------------------
Response (completed in 4.45s):
The query selects the Name, Quantity, and ReorderLevel columns from the Inventory table where the Quantity is less than the ReorderLevel. This will return all inventory items that are below their reorder level.

Based on the results, the only inventory item curren

**Key Observation:**

When we compare this to our earlier test of different models using the basic `invoke_model` API, we can see a significant advantage of the Converse API: **we don't have to modify our code when switching between model families**. The Converse API provides a consistent interface across all supported models.

This is a major benefit for production applications, as it allows you to:
1. Experiment with different models during development
2. Switch models easily based on cost, performance, or capability needs
3. Use different models for different types of questions
4. Upgrade to newer models as they become available

All without requiring significant code changes!

## Building an End-to-End MES Chatbot

Finally, let's put everything together into a complete chatbot function that simulates how this would work in a real-world application. This function will:

1. Take a user's question
2. Process it through our question-answering pipeline
3. Display the answer and relevant metrics

This represents what would be implemented in a complete chatbot application.

In [19]:
def mes_chatbot(question, model_id="anthropic.claude-3-haiku-20240307-v1:0", temperature=0.1):
    """
    Simulates the complete MES chatbot workflow
    
    Parameters
    ----------
    question : str
        The user's question about the MES
    model_id : str
        The model ID to use
    temperature : float
        The temperature parameter for the model
        
    Returns
    -------
    dict
        The complete result including all intermediary steps
    """
    print(f"🤖 MES Chatbot")
    print(f"==================")
    print(f"Model: {model_id}")
    print(f"Temperature: {temperature}")
    print(f"==================\n")
    
    print(f"User: {question}\n")
    
    # Start timer
    start_time = time.time()
    
    # Call the question answering function
    result = ask_mes_question(question, model_id, temperature)
    
    # Extract tool outputs for display
    sql_queries = []
    for tool_response in result["tool_responses"]:
        if tool_response["type"] == "execute_sql" and tool_response.get("success", False):
            sql_queries.append({
                "query": tool_response["sql_query"],
                "rows": tool_response.get("row_count", 0),
                "execution_time": tool_response.get("execution_time", 0)
            })
    
    # Print summary
    print("\n==================")
    print(f"✓ Total execution time: {result['elapsed_time']:.2f}s")
    print(f"✓ SQL queries executed: {len(sql_queries)}")
    
    for i, query in enumerate(sql_queries):
        print(f"  - Query {i+1}: {query['rows']} rows in {query['execution_time']:.2f}s")
    
    print("==================\n")
    
    return result

Let's test our complete chatbot with a business question:

In [20]:
# Test with a business question about quality
mes_chatbot("What's our overall defect rate across all products?")

🤖 MES Chatbot
Model: anthropic.claude-3-haiku-20240307-v1:0
Temperature: 0.1

User: What's our overall defect rate across all products?

Question: What's our overall defect rate across all products?
--------------------------------------------------
Tool request received: get_schema, ID: tooluse_F3gdA_zdQfyxdoS_8kO8uw
Returning cached schema

Schema retrieved: 14 tables, 127 columns
Tool request received: execute_sql, ID: tooluse_A4dBojSKR-CPhVTBM8RHqg
Executing SQL query: SELECT 
  ROUND(AVG(DefectRate), 4) AS overall_defect_rate
FROM QualityControl;
Query executed successfully: 1 rows returned in 3.1ms

Query results (1 rows):
   overall_defect_rate
0                 0.03
--------------------------------------------------
Response (completed in 4.41s):
The query calculates the overall defect rate by taking the average of the DefectRate column from the QualityControl table. This provides the overall defect rate across all products and inspections.

The key steps are:
1. Select the Def

{'question': "What's our overall defect rate across all products?",
 'response': 'The query calculates the overall defect rate by taking the average of the DefectRate column from the QualityControl table. This provides the overall defect rate across all products and inspections.\n\nThe key steps are:\n1. Select the DefectRate column from the QualityControl table.\n2. Use the AVG() aggregate function to calculate the average of the DefectRate values.\n3. Round the result to 4 decimal places to get the overall defect rate.\n\nThis gives us the overall defect rate of 0.0300 or 3.00% across all products.',
 'tool_responses': [{'type': 'get_schema',
   'data': {'total_tables': 14,
    'total_columns': 127,
    'schema': {'Products': {'columns': [{'name': 'ProductID',
        'type': 'INTEGER',
        'notnull': True,
        'pk': True},
       {'name': 'Name', 'type': 'VARCHAR', 'notnull': True, 'pk': False},
       {'name': 'Description',
        'type': 'VARCHAR',
        'notnull': Fal

Let's try one more complex question that requires more advanced analysis:

In [21]:
# Test with a complex question that requires analysis
mes_chatbot("Which work center has the highest production rate and what products are predominantly made there?")

🤖 MES Chatbot
Model: anthropic.claude-3-haiku-20240307-v1:0
Temperature: 0.1

User: Which work center has the highest production rate and what products are predominantly made there?

Question: Which work center has the highest production rate and what products are predominantly made there?
--------------------------------------------------
Tool request received: get_schema, ID: tooluse_MqCUQFtgQmyW7LS_FV3j4A
Returning cached schema

Schema retrieved: 14 tables, 127 columns
Tool request received: execute_sql, ID: tooluse_fIN_RcaOTDKEUw9dSo8XGA
Executing SQL query: SELECT 
  wc.Name AS WorkCenterName,
  wc.Capacity AS Capacity,
  wc.CapacityUOM,
  p.Name AS ProductName,
  p.StandardProcessTime
FROM WorkCenters wc
JOIN WorkOrders wo ON wc.WorkCenterID = wo.WorkCenterID
JOIN Products p ON wo.ProductID = p.ProductID
GROUP BY wc.WorkCenterID
ORDER BY wc.Capacity DESC
LIMIT 1;
Query executed successfully: 1 rows returned in 5.22ms

Query results (1 rows):
     WorkCenterName  Capacity  Capaci

{'question': 'Which work center has the highest production rate and what products are predominantly made there?',
 'response': 'The work center with the highest production rate is the "Wheel Production" work center, which has a capacity of 30 wheels per hour. The predominant product made at this work center is "Bearings", which has a standard process time of 1.36 hours.',
 'tool_responses': [{'type': 'get_schema',
   'data': {'total_tables': 14,
    'total_columns': 127,
    'schema': {'Products': {'columns': [{'name': 'ProductID',
        'type': 'INTEGER',
        'notnull': True,
        'pk': True},
       {'name': 'Name', 'type': 'VARCHAR', 'notnull': True, 'pk': False},
       {'name': 'Description',
        'type': 'VARCHAR',
        'notnull': False,
        'pk': False},
       {'name': 'Category', 'type': 'VARCHAR', 'notnull': False, 'pk': False},
       {'name': 'Cost', 'type': 'FLOAT', 'notnull': True, 'pk': False},
       {'name': 'StandardProcessTime',
        'type': 'FL

## Conclusion

In this notebook, we've built a complete natural language interface for querying a Manufacturing Execution System. This approach allows anyone in your organization to access manufacturing data without needing to understand SQL or database schemas.

**Key learnings:**

1. **Foundation Model Integration**: We used Amazon Bedrock's Converse API to connect foundation models to industrial data sources.

2. **Tool Use Pattern**: We implemented a tool use pattern that allows the model to:
   - Retrieve database schema information
   - Generate and execute SQL queries
   - Process the results into natural language responses

3. **Model Flexibility**: The Converse API allows us to easily switch between different models without changing our application code.

4. **End-to-End Solution**: We built a complete chatbot that takes a question and returns a clear, informative answer based on up-to-date manufacturing data.

This pattern can be extended to other industrial data stores, including:
- Other MES systems
- ERP systems
- SCADA systems
- Data historians
- Industrial IoT platforms
- Custom manufacturing databases

By making industrial data more accessible through natural language interfaces, you can democratize data access across your organization and enable more data-driven decision making at all levels.

## Next Steps

To deploy a more user-friendly interface for this MES chatbot, we've created a simple Streamlit application:

```bash
streamlit run chatbot/Chat.py 
```

If you're running this in Amazon SageMaker Studio JupyterLab, you can access the Streamlit app through the proxy at:
```
https://{domainId}.studio.{region}.sagemaker.aws/jupyterlab/default/proxy/8501/
```
(Make sure to include the trailing forward slash)

In a production environment, you might want to consider:

1. **Authentication and Authorization**: Add user authentication and role-based access control
2. **Caching**: Cache common queries and schema information to improve performance
3. **Monitoring and Logging**: Track usage, performance, and query patterns
4. **Customization**: Tailor the system prompts for specific manufacturing use cases
5. **Integration**: Connect to multiple data sources within your industrial environment

By leveraging foundation models with tool-calling capabilities, you can create powerful natural language interfaces to your industrial data systems, making data more accessible to everyone in your organization.